In [1]:
# European Centre for Disease Control
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

ageData = dt.timedelta(hours=1, minutes=0)
startTime = dt.datetime.now()

try:
    if dataStamp + ageData < dt.datetime.today():
        load = True
        print('Data will be reloaded')
    else:
        load = False
        print('Data already loaded')
except:
    load = True

if load:
    # this is where we read the data from
    print('Loading Data')
    df = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv")
    print('Data Loaded')
    dataStamp = dt.datetime.today()

Loading Data
Data Loaded


In [2]:
# Load the function, issue default of United Kingdom

def get_country(country = 'United_Kingdom'):
    df1 = df.loc[df['countriesAndTerritories'] == country][['countriesAndTerritories', 'dateRep', 'cases', 'deaths']]    # Take just the columns we want
    #df1['dateRep'] = pd.to_datetime(df['dateRep'])
    df1['dateRep'] = df1.apply(lambda row: pd.to_datetime(row['dateRep'], dayfirst = True), axis=1)    # Convert the date string to valid date data
    df1.sort_values(by = 'dateRep', inplace = True)    # Sort the data incase
    df1['cases'] = df1['cases'].apply(lambda x: x if x > 0 else 0 )    # Some countries have dodgy negative values - set them to Zero
    df1['cases_7_days'] = df1.iloc[:,2].rolling(window=7).mean()    # This adds the rolling 7 day average
    df1['deaths'] = df1['deaths'].apply(lambda x: x if x > 0 else 0 )    # Some countries have dodgy negative values - set them to Zero
    df1['deaths_7_days'] = df1.iloc[:,3].rolling(window=7).mean()    # add the 7 day rolling average
    df1['cases_stdev'] = df1.iloc[:,2].rolling(window=21).std()    # get the standard deviation 
    df1['deaths_stdev'] = df1.iloc[:,3].rolling(window=21).std()    # get the standard deviation
    return df1    # Return the dataframe
#df1 = get_country('Germany')
#df1 = get_country()
#df1[-30:]
print('Function Loaded')

Function Loaded


In [3]:
i = 0    # counter for colors
j = 1    # counter for colors
w = 0.9    # width of bars
d = 35    # number of days for the right hand charts
xrot = 30    # number of degrees to rotatet eh xticks

dataTime = dataStamp.strftime("%d/%b/%Y, %H:%M:%S")

# Now we are going to loop through the list of countries and create 4 charts and an image file for each. 
for country in ['United_Kingdom', 'Germany', 'France', 'Spain', 'United_States_of_America',
                'India', 'Ireland']:

    df1 = get_country(country)    # Get the country data
    plt.rcParams["figure.figsize"] = (18,9)    # Create the Plot size
    fig , ax = plt.subplots(2,2)    # Create the plot space
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.35) # add horizontal spacing

    plt.subplot(221)    # position top left
    plt.grid('on')    # turn on grid lines
    plt.xticks(rotation=xrot)    # rotate the xticks values to make them readable
    plt.bar('dateRep', 'cases', color='C{}'.format(i%10), data=df1, width=w)    # Create the bar values for the daily data
    plt.plot('dateRep', 'cases_7_days', color='C{}'.format(j%10), data=df1, linewidth=3)    # Draw the line on top
    plt.title('{} - Cases all-time'.format(country))    # Set the title
    
    plt.subplot(223)    # position bottom left
    plt.grid('on')    # turn on grid lines
    plt.xticks(rotation=xrot)    # rotate the xticks values to make them readable
    plt.bar('dateRep', 'deaths', color='C{}'.format((i)%10), data=df1, width=w)
    plt.plot('dateRep', 'deaths_7_days', color='C{}'.format((j)%10), data=df1, linewidth=3)
    plt.title('{} - Deaths all-time'.format(country))
    
    df1 = df1.loc[df1['dateRep'] > pd.to_datetime('today') - pd.offsets.DateOffset(days=d)]    # this cuts the last d days for more recent data view
    x = []    # create an empty list for the x values
    for day in df1.dateRep.tolist():    # now for each one convert the timestamp to a string
        x.append(day.strftime('%b-%d'))    # string formatting

    plt.subplot(222)    # position top right
    plt.grid('on')    # turn on grid lines

    y1 = df1.cases.tolist()    # set the Y values
    y2 = df1.cases_7_days.tolist()    # set the Y values
    ystd = df1.cases_stdev.tolist()    # set the error marging the standard deviation value

    plt.bar(x,y1, color='C{}'.format((i)%10), width=w, yerr=ystd)    # plot the x and y1, with color and width
    plt.xticks(x[::3], rotation=30)    # Show ticks every 3 days
    plt.plot(x,y2, color='C{}'.format((j)%10), linewidth=3)    # plot the x and y2, with color and line thickness
    plt.title('{} - Cases last {} days'.format(country, d))

    plt.subplot(224)    # position bottom right
    plt.grid('on')    # turn on grid lines
 
    y1 = df1.deaths.tolist()    # set the Y values
    y2 = df1.deaths_7_days.tolist()    # set the Y values
    ystd = df1.deaths_stdev.tolist()    # set the error marging the standard deviation value

    plt.bar(x,y1, color='C{}'.format((i)%10), width=w, yerr=ystd)    # plot the x and y1, with color and width
    plt.xticks(x[::3], rotation=30)    # Show ticks every 3 days
    plt.plot(x,y2, color='C{}'.format((j)%10), linewidth=3)    # plot the x and y2, with color and line thickness
    plt.title('{} - Deaths last {} days'.format(country, d))    # set the title
    
    fig.legend(['7 day average', 'Daily'], fontsize=12, loc=8)    # set the legend on
    fig.suptitle('{} - {}'.format(country,dataTime), fontsize=16)
    i += 2    # move the color counter
    j += 2    # move the color counter
    plt.show()    # show the result
    
    fig.savefig('{}.png'.format(country))    # save the image file of the charts

KeyError: "['deaths', 'cases'] not in index"

In [ ]:
x
dates = []
for day in df1.dateRep[::3]:
#    print(day.strftime('%Y-%b-%d'))
    dates.append(day.strftime('%Y-%b-%d'))
dates
x

In [ ]:
plt.rcParams["figure.figsize"] = (16,6)    # Create the Plot size
#plt.grid('on')    # turn on grid lines
#plt.xticks(rotation=xrot)    # rotate the xticks values to make them readable
plt.bar('dateRep', 'deaths', color='C{}'.format((i)%10), data=df1, width=w)
#plt.plot('dateRep', 'deaths_7_days', color='C{}'.format((j)%10), data=df1, linewidth=3)
#plt.xticks(np.arange(plt.xlim()[0], plt.xlim()[1], 5))
plt.title('{} - Deaths last {} days'.format(country, d))#
plt.show()

In [ ]:
x = df1.dateRep.tolist()
y1 = df1.deaths.tolist()
y2 = df1.deaths_7_days.tolist()

plt.bar(x,y1)
plt.xticks(x[::3], rotation=30)
plt.plot(x,y2)
plt.show


In [ ]:
y2

In [ ]:
pd.to_datetime('today') - pd.offsets.DateOffset(days=60)

In [ ]:
df1

In [ ]:
0%10

In [ ]:
df1 = get_country()
df1[-30:]

In [ ]:
lista = pd.unique(df[['countriesAndTerritories', 'popData2019']].values.ravel())
for i in range(0, len(lista) - 1, 2):
    print(lista[i:i+2])

In [ ]:
len(lista)
lista[0]

In [ ]:
pd.unique(df['countriesAndTerritories'])

In [ ]:
print(dataStamp.strftime("%d/%b/%Y, %H:%M:%S"))

In [ ]:
#(dt.datetime.now() - startTime).strftime('%M minutes, %S seconds')
timeTaken = (dt.datetime.now() - startTime).total_seconds()
print("Started : {} \nEnded   : {}\nTook    : {} minutes, {} seconds".format(startTime, dt.datetime.now(), int(timeTaken // 60), int(timeTaken % 60)))

In [ ]:
timeTaken / 60 
int(timeTaken % 60)

In [ ]:
#np.arange(df1.dateRep)
df1

In [5]:
df.columns

Index(['dateRep', 'year_week', 'cases_weekly', 'deaths_weekly',
       'countriesAndTerritories', 'geoId', 'countryterritoryCode',
       'popData2019', 'continentExp',
       'notification_rate_per_100000_population_14-days'],
      dtype='object')